In [15]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np


In [16]:
# Gather data
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)

log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns=['PRICE'])

c:\Users\Sulav Bhandari\.conda\envs\mlcc\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California ho

In [17]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

property_stats = features.mean().values.reshape(1, 11)

In [18]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

In [19]:
def get_log_estimate(
    nr_rooms, students_per_classroom, next_to_river=False, high_confidence=True):
    """Estimate the price of a property in Boston.

    Keyword arguments:
    None

    Returns:
    Estimated price of the property.
    """

    # Configure property
    property_stats[0][ZN_IDX] = next_to_river
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom

    # Make prediction
    log_estimate = regr.predict(property_stats)[0][0]

    # Calc range
    if high_confidence:
        upper_bound = log_estimate + 2 * RMSE
        lower_bound = log_estimate - 2 * RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68

    return log_estimate, upper_bound, lower_bound, interval


In [20]:
get_log_estimate(3, 20, next_to_river=True, high_confidence=False)

c:\Users\Sulav Bhandari\.conda\envs\mlcc\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


(2.667613460744845, 2.8551255959419755, 2.480101325547715, 68)

In [21]:
np.median(boston_dataset.target)

21.2

In [27]:
# Writing a program that converts the log price estimate
# using 1970s price as well as teh upper and lower bounds
# of today's prices.Rounding values to the nearest 1000 dollars.
Zillow_median_price = 583.3
scale_factor = Zillow_median_price / np.median(boston_dataset.target)

log_est, upper, lower, conf = get_log_estimate(
    9, students_per_classroom=15, next_to_river=False, high_confidence=False
)

# Convert to today's dollars
dollar_est = np.e**log_est * 1000 * scale_factor
dollar_upper = np.e**upper * 1000 * scale_factor
dollar_lower = np.e**lower * 1000 * scale_factor

# Round the value to nearest thousand
rounded_est = np.around(dollar_est, -3)
rounded_upper = np.around(dollar_upper, -3)
rounded_lower = np.around(dollar_lower, -3)

print(f"The Estimated property value is {rounded_est}.")
print(f"At {conf}% confidence the valuation range is")
print(f"USD {rounded_lower} at the lower end to USD {rounded_upper} at the high end.")



The Estimated property value is 823000.0.
At 68% confidence the valuation range is
USD 682000.0 at the lower end to USD 992000.0 at the high end.


c:\Users\Sulav Bhandari\.conda\envs\mlcc\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [41]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):

    """Estimate the price of a property in Boston.

    Keyword arguments:
    rm -- number of rooms in the property.
    ptratio -- number of students per teacher in the classroom for the school in the area.
    chas -- True if the property is next to the river, False otherwise.
    large_range -- True for a 95% prediction interval, False for a 68% interval.

    """

    if rm < 1 or ptratio < 1:
        print("That is unrealistic. Try again.")
        return

    log_est, upper, lower, conf = get_log_estimate(
        rm,
        students_per_classroom=ptratio,
        next_to_river=chas,
        high_confidence=large_range,
    )

    # Convert to today's dollars
    dollar_est = np.e**log_est * 1000 * scale_factor
    dollar_upper = np.e**upper * 1000 * scale_factor
    dollar_lower = np.e**lower * 1000 * scale_factor

    # Round the value to nearest thousand
    rounded_est = np.around(dollar_est, -3)
    rounded_upper = np.around(dollar_upper, -3)
    rounded_lower = np.around(dollar_lower, -3)

    print(f"The Estimated property value is {rounded_est}.")
    print(f"At {conf}% confidence the valuation range is")
    print(
        f"USD {rounded_lower} at the lower end to USD {rounded_upper} at the high end."
    )


In [43]:
get_dollar_estimate(rm=6, ptratio=20, chas=True)

The Estimated property value is 520000.0.
At 95% confidence the valuation range is
USD 358000.0 at the lower end to USD 757000.0 at the high end.


c:\Users\Sulav Bhandari\.conda\envs\mlcc\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [48]:
# As we have created a module named valuation.py with all the code in this notebook, we can import it and use it

import valuation

valuation.get_dollar_estimate(rm=6, ptratio=20, chas=True)


The Estimated property value is 520000.0.
At 95% confidence the valuation range is
USD 358000.0 at the lower end to USD 757000.0 at the high end.


c:\Users\Sulav Bhandari\.conda\envs\mlcc\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
